# Image Classification

## About
- Fine-tune ViT on the Food-101 dataset to classify a food item in an image.
- Utilise fine-tuned model for inference.

## Loading Food-101 Dataset

In [ ]:
from datasets import load_dataset

food = load_dataset('ethz/food101', split='train[:5000]')

food = food.train_test_split(test_size=0.2)
food['train'][0]

c:\Users\YashPrasad\miniconda3\envs\pyenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import importlib.metadata

print(importlib.metadata.version("pandas"))
print(importlib.metadata.version("fsspec"))

None
2025.7.0
